In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sklearn.model_selection import train_test_split
from sentence_transformers.evaluation import InformationRetrievalEvaluator

# Load CSV
df = pd.read_csv("Copy of 500 lable แล้ว ไม่มีปริมาณ superclean + มีของคาวของหวาน + queryครบ - Sheet1-3.csv")

# Rename columns
df = df.rename(columns={
    "วัตถุดิบ_ไม่มีปริมาณ": "ingredients",
    "ชื่ออาหาร": "dish"
})

# Drop missing values
# Ensure column names in dropna match the dataframe
df = df.dropna(subset=["ingredients", "dish", "query2", "query3"])

# Split into train/temp (80/20)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Split temp into val/test (50/50 of temp)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Add query 1 (the first val example) into training set
query1_row = val_df.iloc[0]
train_df = pd.concat([
    train_df,
    pd.DataFrame({
        "ingredients": [query1_row["ingredients"]],
        "dish": [query1_row["dish"]],
        "query2": [query1_row["query2"]],
        "query3": [query1_row["query3"]],
    })
], ignore_index=True)

val_df = val_df.drop(val_df.index[0]).reset_index(drop=True)

# ----- 🔁 Prepare augmented training samples -----
def prepare_training_samples(df):
    anchors = []
    positives = []
    for _, row in df.iterrows():
        anchors.extend([
            row["ingredients"],
            row["query2"],
            row["query3"]
        ])
        positives.extend([row["dish"]] * 3)
    return anchors, positives

train_anchors, train_positives = prepare_training_samples(train_df)

# Validation and test keep using only 'ingredients'
val_samples = list(zip(val_df["ingredients"], val_df["dish"]))
test_samples = list(zip(test_df["ingredients"], test_df["dish"]))

# Convert to HF datasets
def convert_to_dict(samples):
    return Dataset.from_dict({
        "anchor": [a for a, b in samples],
        "positive": [b for a, b in samples],
    })

train_dataset = Dataset.from_dict({
    "anchor": train_anchors,
    "positive": train_positives
})
val_dataset = convert_to_dict(val_samples)
test_dataset = convert_to_dict(test_samples)

# Load model
model = SentenceTransformer(
    'paraphrase-multilingual-mpnet-base-v2',
    model_card_data=SentenceTransformerModelCardData(
        language="th",
        license="apache-2.0",
        model_name="Thai Food Ingredients → Dish Prediction",
    )
)

# Evaluation
val_queries = {f'q{i}': val_dataset['anchor'][i] for i in range(len(val_dataset))}
val_corpus = {f'd{i}': val_dataset['positive'][i] for i in range(len(val_dataset))}
val_relevant_docs = {f'q{i}': {f'd{i}'} for i in range(len(val_dataset))}

evaluator = InformationRetrievalEvaluator(
    queries=val_queries,
    corpus=val_corpus,
    relevant_docs=val_relevant_docs,
    name='thai-food-eval',
    show_progress_bar=True,
    precision_recall_at_k=[1, 3, 5]
)

# Loss function
loss = MultipleNegativesRankingLoss(model)

# Training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="models/thai-food-mpnet-new-v8",
    num_train_epochs=7,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    learning_rate=5e-6,
    warmup_ratio=0.1,
    fp16=False,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_strategy="steps",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    run_name="thai-food-retriever-tuned",
)

# Train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=loss,
    evaluator=evaluator,
)
trainer.train()

# Save model
model.save_pretrained("models/thai-food-mpnet-new-v8")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: Currently logged in as: chanisorn-siri (chanisorn-siri-bodindecha) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Thai-food-eval Cosine Accuracy@1,Thai-food-eval Cosine Accuracy@3,Thai-food-eval Cosine Accuracy@5,Thai-food-eval Cosine Accuracy@10,Thai-food-eval Cosine Precision@1,Thai-food-eval Cosine Precision@3,Thai-food-eval Cosine Precision@5,Thai-food-eval Cosine Recall@1,Thai-food-eval Cosine Recall@3,Thai-food-eval Cosine Recall@5,Thai-food-eval Cosine Ndcg@10,Thai-food-eval Cosine Mrr@10,Thai-food-eval Cosine Map@100
1,1.516000,1.667425,0.395833,0.666667,0.750000,0.854167,0.395833,0.222222,0.150000,0.395833,0.666667,0.750000,0.619401,0.544527,0.552980
2,0.893200,1.266092,0.416667,0.687500,0.833333,0.895833,0.416667,0.229167,0.166667,0.416667,0.687500,0.833333,0.657152,0.580233,0.585154
3,0.919200,1.090231,0.520833,0.750000,0.854167,0.895833,0.520833,0.250000,0.170833,0.520833,0.750000,0.854167,0.723250,0.666518,0.671825
4,0.679800,0.984286,0.604167,0.791667,0.854167,0.916667,0.604167,0.263889,0.170833,0.604167,0.791667,0.854167,0.765706,0.717130,0.721777
5,0.524600,0.956241,0.625000,0.812500,0.875000,0.916667,0.625000,0.270833,0.175000,0.625000,0.812500,0.875000,0.771473,0.724479,0.729034
6,0.493000,0.947175,0.625000,0.833333,0.875000,0.916667,0.625000,0.277778,0.175000,0.625000,0.833333,0.875000,0.777170,0.731481,0.736376
7,0.493400,0.947075,0.625000,0.854167,0.875000,0.937500,0.625000,0.284722,0.175000,0.625000,0.854167,0.875000,0.784637,0.735301,0.738302


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Corpus Chunks: 100%|██████████| 1/1 [00:02<00:00,  2.13s/it]


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:02<00:00,  2.12s/it]


In [ ]:
# Save test set to CSV
test_df[["ingredients", "dish"]].to_csv("test_set_for_google_sheets.csv", index=False)
print("✅ Test set saved as 'test_set_for_google_sheets.csv'")


✅ Test set saved as 'test_set_for_google_sheets.csv'


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

# Save model locally
model.save_pretrained("models/thai-food-mpnet-new-v8")

# (Optional) Create a model card
trainer.create_model_card(
    model_name="thai_food_prediction1",
    language="th",
    license="apache-2.0",
    tags=["thai", "semantic-search", "food", "ingredients", "retrieval", "sentence-transformers"]
)

# Push model to Hugging Face Hub
trainer.push_to_hub(repo_name="Chanisorn/thai_food_prediction1")


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

events.out.tfevents.1749386969.7bcd77a398f2.1700.1:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

events.out.tfevents.1749387095.7bcd77a398f2.58986.0:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

events.out.tfevents.1749373481.7bcd77a398f2.1700.0:   0%|          | 0.00/27.3k [00:00<?, ?B/s]

Upload 8 LFS files:   0%|          | 0/8 [00:00<?, ?it/s]

events.out.tfevents.1749398840.7bcd77a398f2.102910.0:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Chanisorn/thai-food-mpnet-new-v8/commit/01f5f36079ec3c939f38ba55dc5acbac27b57f10', commit_message='End of training', commit_description='', oid='01f5f36079ec3c939f38ba55dc5acbac27b57f10', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Chanisorn/thai-food-mpnet-new-v8', endpoint='https://huggingface.co', repo_type='model', repo_id='Chanisorn/thai-food-mpnet-new-v8'), pr_revision=None, pr_num=None)

In [ ]:
!pip install -U sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

ยังไม่ pythai

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Load the model
model = SentenceTransformer("Chanisorn/thai-food-mpnet-new-v8")

# Load and clean the dataset
df = pd.read_csv("a.csv")
df = df.rename(columns={"วัตถุดิบ": "ingredients", "ชื่ออาหาร": "dish"})
df = df.dropna(subset=["ingredients", "dish"])

# Encode dish names
dish_texts = df["dish"].tolist()
dish_embeddings = model.encode(dish_texts, convert_to_tensor=True)

# Input sets (ingredient queries)
input_sets = [
    "อยากกินอาหารที่แบบนัวๆ ใส่กุ้งเยอะๆ มีฟักทอง เผ็ด",
    "มีผงโกโก้ทำอะไรกินดี",

]

# Inference loop for top-3 matches
for user_input in input_sets:
    input_embedding = model.encode([user_input], convert_to_tensor=True)
    cos_sim = cosine_similarity(input_embedding.cpu(), dish_embeddings.cpu())[0]

    top_indices = np.argsort(cos_sim)[::-1][:3]

    print(f"\n🔍 Input Ingredients: {user_input}")
    for rank, idx in enumerate(top_indices, 1):
        matched_dish = df.iloc[idx]["dish"]
        matched_ingredients = df.iloc[idx]["ingredients"]
        similarity_score = cos_sim[idx]

        print(f"\n🥇 Rank {rank}")
        print(f"✅ Matched Dish: {matched_dish}")
        print(f"🧂 Ingredients: {matched_ingredients}")
        print(f"📏 Similarity Score: {similarity_score:.4f}")




🔍 Input Ingredients: อยากกินอาหารที่แบบนัวๆ ใส่กุ้งเยอะๆ มีฟักทอง เผ็ด

🥇 Rank 1
✅ Matched Dish: แกงบวดฟักทอง
🧂 Ingredients: ฟักทอง:, 100 g
น้ำตาลทราย:, 3 ช้อนโต๊ะ
กะทิ:, 100 มล.
เกลือ: เล็กน้อย
📏 Similarity Score: 0.6271

🥇 Rank 2
✅ Matched Dish: ผัดฟักทอง
🧂 Ingredients: ฟักทองหั่นชิ้น, 200 กรัม
ไข่ไก่, 1 ฟอง
น้ำเปล่า, 1 ถ้วย
กระเทียมสับหยาบ, 1 ช้อนโต๊ะ
ซีอิ๊ว low sodium, 1 ช้อนโต๊ะ
พริกไทยป่น, 1 ช้อนโต๊ะ
ใบโหระพา, 10 ใบ
น้ำมันสำหรับผัด, 1 ช้อนชา
📏 Similarity Score: 0.5742

🥇 Rank 3
✅ Matched Dish: ผัดฟักทองใส่ไข่ นิ่ม ฟิน
🧂 Ingredients: ฟักทอง:, 500 กรัม
ไข่ไก่:, 2 ฟอง
น้ำตาล:, 1 ช้อนโต๊ะ
น้ำปลา:, 1/2 ช้อน
แมกกี้: เหยาะๆให้หอม
น้ำมันหอย:, 2 ช้อนโต๊ะ
กระเทียม:, 4 กลีบ
📏 Similarity Score: 0.5578

🔍 Input Ingredients: มีผงโกโก้ทำอะไรกินดี

🥇 Rank 1
✅ Matched Dish: โกโก้หนึบ
🧂 Ingredients: นมข้นหวาน, 200g
ผงโกโก้(ผงโอวัลตินก็ได้), 100g
📏 Similarity Score: 0.8321

🥇 Rank 2
✅ Matched Dish: โกโก้มัทฉะ
🧂 Ingredients: ผงมัทฉะ, 20 กรัม
นมสำหรับชงชาเขียว, 2 ถ้วยตวง
น้ำตาลสำหรับชงชาเขียว ¼ ถ้วย

ลองใช้ pythainlp 

In [ ]:
!pip install pythainlp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 58.5 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# ===== 🔤 Pythainlp เพื่อ clean ข้อความ =====
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_stopwords

def clean_text_thai(text):
    text = normalize(text)  # Normalize เช่น ไม้ยมก
    tokens = word_tokenize(text, keep_whitespace=False)  # ตัดคำ
    stopwords = set(thai_stopwords())
    tokens = [t for t in tokens if t not in stopwords]  # ลบ stopwords
    return " ".join(tokens)

# ===== 🧠 Load Model =====
model = SentenceTransformer("Chanisorn/thai-food-mpnet-new-v8")

# ===== 📄 Load Dataset =====
df = pd.read_csv("a.csv")
df = df.rename(columns={"วัตถุดิบ": "ingredients", "ชื่ออาหาร": "dish"})
df = df.dropna(subset=["ingredients", "dish"])

# ===== 💾 Encode Dish Embeddings =====
dish_texts = df["dish"].tolist()
dish_embeddings = model.encode(dish_texts, convert_to_tensor=True)

# ===== 🍳 Input จากผู้ใช้ =====
input_sets = [
    "อยากกินอาหารที่แบบนัวๆ ใส่กุ้งเยอะๆ มีฟักทอง เผ็ด",
    "มีผงโกโก้ทำอะไรกินดี",
]

# ===== 🔍 Matching Loop =====
for user_input in input_sets:
    cleaned_input = clean_text_thai(user_input)
    input_embedding = model.encode([cleaned_input], convert_to_tensor=True)
    cos_sim = cosine_similarity(input_embedding.cpu(), dish_embeddings.cpu())[0]

    top_indices = np.argsort(cos_sim)[::-1][:3]

    print(f"\n🔍 Input: {user_input}")
    print(f"🧹 Cleaned: {cleaned_input}")
    for rank, idx in enumerate(top_indices, 1):
        matched_dish = df.iloc[idx]["dish"]
        matched_ingredients = df.iloc[idx]["ingredients"]
        similarity_score = cos_sim[idx]

        print(f"\n🥇 Rank {rank}")
        print(f"✅ Matched Dish: {matched_dish}")
        print(f"🧂 Ingredients: {matched_ingredients}")
        print(f"📏 Similarity Score: {similarity_score:.4f}")



🔍 Input: อยากกินอาหารที่แบบนัวๆ ใส่กุ้งเยอะๆ มีฟักทอง เผ็ด
🧹 Cleaned: กิน อาหาร นัว ใส่ กุ้ง ฟักทอง เผ็ด

🥇 Rank 1
✅ Matched Dish: แกงบวดฟักทอง
🧂 Ingredients: ฟักทอง:, 100 g
น้ำตาลทราย:, 3 ช้อนโต๊ะ
กะทิ:, 100 มล.
เกลือ: เล็กน้อย
📏 Similarity Score: 0.6868

🥇 Rank 2
✅ Matched Dish: ผัดฟักทอง
🧂 Ingredients: ฟักทองหั่นชิ้น, 200 กรัม
ไข่ไก่, 1 ฟอง
น้ำเปล่า, 1 ถ้วย
กระเทียมสับหยาบ, 1 ช้อนโต๊ะ
ซีอิ๊ว low sodium, 1 ช้อนโต๊ะ
พริกไทยป่น, 1 ช้อนโต๊ะ
ใบโหระพา, 10 ใบ
น้ำมันสำหรับผัด, 1 ช้อนชา
📏 Similarity Score: 0.5829

🥇 Rank 3
✅ Matched Dish: ผัดฟักทองใส่ไข่ นิ่ม ฟิน
🧂 Ingredients: ฟักทอง:, 500 กรัม
ไข่ไก่:, 2 ฟอง
น้ำตาล:, 1 ช้อนโต๊ะ
น้ำปลา:, 1/2 ช้อน
แมกกี้: เหยาะๆให้หอม
น้ำมันหอย:, 2 ช้อนโต๊ะ
กระเทียม:, 4 กลีบ
📏 Similarity Score: 0.5387

🔍 Input: มีผงโกโก้ทำอะไรกินดี
🧹 Cleaned: ผง โกโก้ ทำ กิน ดี

🥇 Rank 1
✅ Matched Dish: โกโก้หนึบ
🧂 Ingredients: นมข้นหวาน, 200g
ผงโกโก้(ผงโอวัลตินก็ได้), 100g
📏 Similarity Score: 0.8042

🥇 Rank 2
✅ Matched Dish: โกโก้มัทฉะ
🧂 Ingredients: ผงมัทฉะ, 20 กรัม
นมสำห

In [5]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import pandas as pd

# 🔧 Load your pretrained model
model = SentenceTransformer("Chanisorn/thai-food-mpnet-new-v8")

# 📄 Load your test set CSV (with query1–3 and ชื่ออาหาร)
test_df_raw = pd.read_csv("test set 2 - test_set_for_google_sheets.csv")  # update path if needed

# 📌 Expand test set with anchor-positive pairs
test_rows = []

for _, row in test_df_raw.iterrows():
    dish = row["dish"]
    for col in ["query1 อยากกินอาหารครบ", "query2", "query3"]:
        query = row[col]
        if pd.notna(query):
            test_rows.append({"anchor": query.strip(), "positive": dish.strip()})

test_df = pd.DataFrame(test_rows)

# ✅ Prepare data for evaluator
queries = {f"q{i}": row["anchor"] for i, row in test_df.iterrows()}
corpus = {f"d{i}": row["positive"] for i, row in test_df.iterrows()}
relevant_docs = {f"q{i}": {f"d{i}"} for i in range(len(test_df))}

# 🧪 Evaluate
evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    name="thai-food-test-eval",
    show_progress_bar=True,
    precision_recall_at_k=[1, 3, 5]
)

evaluator(model)



Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:08<00:00,  8.33s/it]


{'thai-food-test-eval_cosine_accuracy@1': 0.2585034013605442,
 'thai-food-test-eval_cosine_accuracy@3': 0.7074829931972789,
 'thai-food-test-eval_cosine_accuracy@5': 0.7891156462585034,
 'thai-food-test-eval_cosine_accuracy@10': 0.8435374149659864,
 'thai-food-test-eval_cosine_precision@1': 0.2585034013605442,
 'thai-food-test-eval_cosine_precision@3': 0.23582766439909295,
 'thai-food-test-eval_cosine_precision@5': 0.15782312925170064,
 'thai-food-test-eval_cosine_recall@1': 0.2585034013605442,
 'thai-food-test-eval_cosine_recall@3': 0.7074829931972789,
 'thai-food-test-eval_cosine_recall@5': 0.7891156462585034,
 'thai-food-test-eval_cosine_ndcg@10': 0.5661307373640255,
 'thai-food-test-eval_cosine_mrr@10': 0.4745653817082389,
 'thai-food-test-eval_cosine_map@100': 0.4812900188022347}